In [1]:
using LinearAlgebra
using Plots
using SparseArrays
using Arpack #Eigenvalues of sparse arrays with eigs()
using DifferentialEquations
# using DiffEqFlux
using Optim
using ForwardDiff
using DelimitedFiles
using LineSearches

In [2]:
#Def parameters
n_cutoff = 8
N = 2 * n_cutoff + 1
NHilbert = N^2
n = sparse(Diagonal(LinRange(-n_cutoff, n_cutoff, N))) #Perhaps implement using StaticArrays
# n = sparse(Diagonal(LinRange(n_cutoff, 3*n_cutoff+1, N))) #Perhaps implement using StaticArrays

Id = sparse(I, N, N)
exp_iPhi = spdiagm(-1 => ones(N - 1));

#Def Hamiltonian parameters
E_J_GHz = 10.0 #scale of E_J in units of h*GHz, h-bar = 1, h = 2pi
E_C = 1/100  #Charging energies
E_J = 1.0 #Josephson energies
phi_ext = 0.995pi

prefactor = 4 * E_C
Kinetic = 2pi * E_J_GHz * prefactor * (kron(n^2, Id) .+ kron(Id, n^2))

Potential1 = -2pi * E_J_GHz * kron(1 / 2 * (exp_iPhi .+ exp_iPhi'), Id) # -E_J1 cos(phi1)
Potential2 = -2pi * E_J_GHz * kron(Id, 1 / 2 * (exp_iPhi .+ exp_iPhi')) # -E_J2 cos(phi2)
Pot3Mat = kron(exp_iPhi, exp_iPhi')
Pot3Const = -2pi * E_J_GHz * E_J / 2

function get_Pot3(Φ_ext = phi_ext)
    M = exp(im * Φ_ext) * Pot3Mat
    return Pot3Const * (M + M')
end

function dPot3_dphi(Φ_ext = phi_ext)
    M = exp(im * Φ_ext) * Pot3Mat
    return (im * Pot3Const) .* (M - M')
end

# Potential3 =  get_Pot3() #-E_J3 cos(phi1 - phi2 + phi_ext)
Potential12 = E_J * (Potential1 + Potential2)
KinPot12 = Kinetic + Potential12
chargecoupling = kron(n, Id);

In [26]:
function limit_func(param,pmin,pmax)
    (pmax - pmin) * (1/(1 + exp(-param))) + pmin
end

function inv_limit_func(param,pmin,pmax)
    # -log( (pmax - pmin  + 2e-8) / (param - pmin + 1e-8) - 1)
    -log( (pmax - pmin) / (param - pmin) - 1)
end

function alpha(p, t)
    Tₐ = 10#limit_func(p[2],0,15)
    Tₚ = 30 - 2*Tₐ#p[3]
    αmin = limit_func(p[1],0.5,1)
    abs_slope = (1 - αmin)/Tₐ
    if t < Tₐ
        return 1 - abs_slope * t
    elseif t > Tₐ + Tₚ
        return αmin + abs_slope * (t - Tₐ - Tₚ)
    else
        return αmin
    end
end

dalphadp(p, t) = ForwardDiff.gradient(p̃ -> alpha(p̃,t), p)

h_times = LinRange(10,20,50+2)[2:end-1]
dt = h_times[2] - h_times[1]
function pulse(p, t)
    if  t < 10 || t > 20
        return 0
    end
    out = 0
    w_start = exp( - 5/(2*dt^2) * (t - 10)^2)
    w_end = exp( - 5/(2*dt^2) * (t - 20)^2)
    w_sum = w_start + w_end
    for (i, ht) in enumerate(h_times)
        w = exp( - 5/(2*dt^2) * (t - ht)^2)
        out += p[i+1] * w
        w_sum += w
    end
    # envelope(t)*out/w_sum
    out/w_sum
end

dpulsedp(p, t) = ForwardDiff.gradient(p̃ -> pulse(p̃, t), p)

dpulsedp (generic function with 1 method)

In [83]:
p = cat([inv_limit_func(0.55,0.5,1)],zeros(50),dims=1) ;

In [84]:
#5 Phis:
N_phis = 5
Pot3s = []
for phi_ext in LinRange(0.994,0.996,N_phis)
    push!(Pot3s,get_Pot3(phi_ext*pi))
end

ψ0 = []
for Pot3 in Pot3s
    ψs = eigvecs(Matrix(KinPot12 + Pot3));
    push!(ψ0,ψs[:,1:2])
end

In [85]:
fu0 = cat(ψ0...,dims=2) ;
sizep = length(p) ;

In [86]:
function f!(du, u, p, t)
    for i in 1:N_phis
        H = KinPot12 + alpha(p,t) .* Pot3s[i] + pulse(p,t) .* chargecoupling
        du[:,2*i-1:2*i] = -im .* (H * u[:,2*i-1:2*i])
    end
end

function b!(du, u, p, t)
    alphasum = 0
    for i in 1:N_phis

        H = KinPot12 + alpha(p,t) .* Pot3s[i] + pulse(p,t) .* chargecoupling

        du[:,2*i-1:2*i] = -im .* (H * u[:,2*i-1:2*i])

        du[:,2*N_phis + 2*i-1: 2*N_phis + 2*i] = -im .* (H * u[:,2*N_phis + 2*i-1: 2*N_phis + 2*i])

        alphasum += real(im * tr(u[:,2*N_phis + 2*i-1: 2*N_phis + 2*i]' * Pot3s[i] * u[:,2*i-1:2*i]))
    end

    du[1:sizep,end] = 2 .* (
        - real(im * tr(u[:,2*N_phis+1:4*N_phis]' * chargecoupling * u[:,1:2*N_phis])) .* dpulsedp(p,t)
        - alphasum .* dalphadp(p,t) )
end

b! (generic function with 1 method)

In [87]:
ls = []
params = []
counter = [0]

grad = zeros(sizep)
loss = [0.0]

function calc(p, last_p, grad, loss)
    if p != last_p
        counter[1] += 1
        copy!(last_p, p)
        forward_prob = ODEProblem(f!, fu0, (0.0, 30.0), p)
        sol_f = solve(forward_prob, p=p, save_everystep=false, reltol=1e-8, abstol=1e-8)

        xT = sol_f.u[end][:,1:2*N_phis]

        λs = []
        for i in 1:N_phis
            push!(λs,0.5 .* cat(ψ0[i][:,2]*ψ0[i][:,2]'*xT[:,2*i-1],ψ0[i][:,1]*ψ0[i][:,1]'*xT[:,2*i],dims=2))
        end
        
        λ = cat(λs...,dims=2)

        ps = zeros(289,1) #only 2*length(p) are possibly used
        bu0 = cat(xT,λ,ps,dims=2);

        backward_prob = ODEProblem(b!, bu0, (30.0, 0.0), p)
        sol_b = solve(backward_prob, p=p, save_everystep=false, reltol=1e-8, abstol=1e-8)

        UT = abs2.(fu0'*xT)
        iFs = []
        for i in 1:N_phis
            push!(iFs,0.5 * (2 - UT[2*i-1,2*i] - UT[2*i,2*i-1]))
        end

        loss[1] = sum(iFs)/N_phis
        grad[:] = real(sol_b.u[end][1:sizep,end])

        push!(ls, copy(iFs))
        push!(params, copy(p))

        if counter[1]%5 == 0
            image = plot(ls,yscale=:log10,ylim=(1e-5,2))
            savefig(image,"tmp_loss.png")

            writedlm("tmp_awg55_lsN5.txt",ls)
            writedlm("tmp_awg55_paramsN5.txt",params)
        end
    end
end


function opt_func(p, last_p, grad, loss)
    calc(p, last_p, grad, loss)
    return loss[1]
end

function get_grad!(grad, p, last_p, loss)
    calc(p, last_p, grad, loss)
    println(ls[end],loss[1])
end

get_grad! (generic function with 1 method)

In [1]:
342*60/160

128.25

In [88]:
initial_p = copy(p)
last_p = similar(initial_p)
# get_grad!(grad, p, last_p, loss)
;

In [ ]:
#342m 160

In [89]:
res = optimize(p -> opt_func(p, last_p, grad, loss),
               (grad, p) -> get_grad!(grad, p, last_p, loss),
               initial_p,BFGS(linesearch=LineSearches.MoreThuente()))

Any[0.9996728457113122, 0.9986382625723776, 0.9974570229396762, 0.9970136911514479, 0.9969187473332897]0.9979401139416207


Any[0.9894761723642668, 0.9883923006724065, 0.9886328112904426, 0.9880102239991826, 0.9827737769226406]0.9874570570497878


Any[0.9952166964604787, 0.9951592020409077, 0.9962595815071192, 0.9978622164105294, 0.9983331264191204]0.9965661645676311


Any[0.999709300748057, 0.9995916123395162, 0.9992520927527219, 0.9992933436885022, 0.9998547066969253]0.9995402112451446


Any[0.9729532382415846, 0.9791896948078683, 0.9842311563968317, 0.9861761401523864, 0.9853788865364284]0.98158582322702

...

Any[2.1765130440720704e-5, 1.1728301005820008e-5, 8.678160228481335e-6, 1.2102817343762062e-5, 2.402669384127254e-5]1.566022057201133e-5


Any[2.1695052364489875e-5, 1.171903548868869e-5, 8.699676847412352e-6, 1.2151660205850234e-5, 2.3989209274100087e-5]1.565092683610825e-5


Any[2.1572590208074338e-5, 1.1682870204920981e-5, 8.730408345702934e-6, 1.223868262101302e-5, 2.3989787373668303e-5]1.5642867750675914e-5


Any[2.1569883962180292e-5, 1.1693721568062365e-5, 8.715872975850747e-6, 1.2212708989545185e-5, 2.3992488310919935e-5]1.5636935161311704e-5


Any[2.156580383205764e-5, 1.1697158227685911e-5, 8.743635247943082e-6, 1.224949080780302e-5, 2.3902431851186012e-5]1.5631703993335134e-5
